In [ ]:
%load_ext autoreload
%autoreload 2

from __future__ import annotations

import geoviews as gv
import geopandas as gpd
import holoviews as hv
import numpy as np
import pandas as pd
import panel as pn
import pygeos
import shapely
import shapely.wkt
import xarray as xr

from holoviews import opts
from holoviews import streams
from holoviews.operation.datashader import dynspread
from holoviews.operation.datashader import datashade
from holoviews.operation.datashader import inspect
from holoviews.operation.datashader import rasterize
from holoviews.streams import PointerXY
from holoviews.streams import Tap

hv.extension("bokeh")

from thalassa import api
from thalassa import normalization
from thalassa import utils

## Normalize datasets

The first thing we need to do is to normalize the dataset to the "thalassa" format.

We do this with the `normalize_dataset()` function.

This function returns an `xr.Dataset` with renamed dimensions/variables + some additional ones. 
Furthermore it splits quad elements to triangles.

In [ ]:
ds = xr.open_dataset("data/schout_20220205.nc")
ds = normalization.normalize_dataset(ds)
ds

## Main API

`thalassa` supports several types of graphs, including:
    
- A visualization of the variables at various timestamps and or layers.
- A visualization of the mesh
- Extraction of timeseries from specific points

The following cell shows the basic usage:

In [ ]:
#variable, layer, timestamp = "salt", 40, ds.time.values[4]
#variable, layer, timestamp = "depth", None, None
variable, layer, timestamp = "elev", None, "max"

# The trimesh is the most basic object. This is what you need to create all the others graphs
# It is on this object that you specify the timestamp and/or the layer.
trimesh = api.create_trimesh(ds=ds, variable=variable, timestamp=timestamp, layer=layer)

# The wireframe is the representation of the mesh
wireframe = api.get_wireframe(trimesh=trimesh)

# The tiles is using the tiling service from Open Street maps
tiles =  api.get_tiles() 

# The raster object is the basic Map that visualizes the variable. You can specify things like the colorbar limits and/or the extents
raster = api.get_raster(trimesh, clim_min=0, clim_max=15)

# The pointer/tap timeseries extract the timeseries of a specific node from the xr.Dataset and visualizet it.
pointer_dmap = api.get_pointer_timeseries(ds=ds, variable=variable, source_raster=raster, layer=layer)
tap_dmap = api.get_tap_timeseries(ds=ds, variable=variable, source_raster=raster, layer=layer)

### Let's visualize!

After you render the layout, move the mouse over the map and click on it. This will fill in the `pointer_dmap` and the `tap_dmap`.

In [ ]:
layout = tiles * raster.opts(width=600, cmap="viridis") 
layout
(pointer_dmap + tap_dmap).opts(shared_axes=False)

## Set the zoom level

In order to set the zoom level, we need to specify a tuple with the `x_range` and `y_range` arguments in `get_raster()`.

In [ ]:
raster2 = api.get_raster(trimesh, clim_min=0, clim_max=15, x_range=(-90, -80), y_range=(20, 30))
tiles * raster2

We can even calcuate the ranges from an existing raster (e.g. to restore the zoom level of a different graph)

To do this we can use the `get_bbox_from_raster()` and `get_?_range_from_bbox()` functions

In [ ]:
bbox = api.get_bbox_from_raster(raster2)
bbox

x_range = api.get_x_range_from_bbox(bbox)
y_range = api.get_y_range_from_bbox(bbox)
x_range
y_range

## Undocumented

In [ ]:
polygon_gdf = api.generate_mesh_polygon(ds)
polygon_gdf